In [ ]:
import pandas as pd
import numpy as np
import json, csv
import itertools as it
import matplotlib.pyplot as plt
import seaborn as sns
import os
from tqdm.notebook import tqdm
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
nlp = spacy.load('en')
import warnings
warnings.filterwarnings("ignore")

In [ ]:

# Import files from google drive
from google.colab import drive
drive.mount('/content/gdrive',force_remount=False)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [ ]:
#change directory
os.chdir('/content/gdrive/My Drive/product_workshop/data')
os.listdir()

['review_corpus.txt',
 'unigram_sentences_all.txt',
 'bigram_sentences_all.txt',
 'trigram_sentences_all.txt',
 'trigram_transformed_reviews_all.txt',
 'trigram_dict_all.dict',
 'trigram_bow_corpus_all.mm.index',
 'trigram_bow_corpus_all.mm',
 'lda_model_all',
 'lda_model_all.expElogbeta.npy',
 'lda_model_all.state',
 'lda_model_all.id2word',
 'ratings_sample.csv',
 'sampled_reviews.csv',
 'sampled_reviews_adj.csv']

In [ ]:
import sys
csv.field_size_limit(sys.maxsize)

131072

In [ ]:
with open('sampled_reviews_adj.csv') as csv_file:
  csv_reader = csv.reader(csv_file, delimiter=',')
  print(next(csv_reader))
  print(next(csv_reader))
  print(next(csv_reader))
  print(next(csv_reader))
  print(next(csv_reader))

['', 'overall', 'verified', 'reviewTime', 'reviewerID', 'asin', 'style', 'reviewerName', 'reviewText', 'summary', 'unixReviewTime', 'vote', 'image', 'title', 'char_count', 'sum_reviews', 'review_adjectives']
['0', '5.0', 'False', '06 19, 2000', 'A9L3BUKDG2XNM', 'B000068MBW', "{'Format:': ' VHS Tape'}", 'Amazon Customer', 'you will watch this movie and be unable to believe that it was made for television it is so well done a labor of love for director jim abrahams whose own son suffered from epilepsy helped by alternative medicine this is the story of a farmer ward and his wife the ever wonderful streep making a rare television appearance whose youngest son robbie adkins developes epilepsy michael yarmush also fares well in his role as robbies older brother frank telling of a childs illness and its profound effect on a family', 'Realistic and Gripping!', '961372800', '10.0', '', '"...First Do No Harm"', '532.0', '80.0', "['unable', 'own', 'alternative', 'wonderful', 'rare', 'profound']"

In [ ]:
%%time
df_cleaned = pd.read_csv('sampled_reviews_adj.csv', converters={'review_adjectives': eval})

CPU times: user 4.51 s, sys: 574 ms, total: 5.08 s
Wall time: 6.22 s


In [ ]:
data_words = df_cleaned.review_adjectives
data_words[0]

['unable', 'own', 'alternative', 'wonderful', 'rare', 'profound']

In [ ]:
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models.ldamulticore import LdaMulticore

In [ ]:
id2word = corpora.Dictionary(data_words)

In [ ]:
corpus = [id2word.doc2bow(text) for text in data_words]

In [ ]:
print(corpus[:2])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1)], [(6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1)]]


In [ ]:
%%time
lda_model = LdaMulticore(corpus=corpus, 
                         id2word=id2word,
                         num_topics=50,
                         random_state=100,
                         chunksize=100,
                         passes=10,
                         per_word_topics=True,
                         workers = 3)

CPU times: user 41min 52s, sys: 3min 34s, total: 45min 27s
Wall time: 47min 5s


In [ ]:
!pip install pyLDAvis

     |████████████████████████████████| 1.6MB 2.7MB/s 
     |████████████████████████████████| 552kB 11.6MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-2.1.2-py2.py3-none-any.whl size=97711 sha256=0161f0c83be666f0885de63a6f0b7d8551102d9cc1a381e9ae71d56620a1d270
  Stored in directory: /root/.cache/pip/wheels/98/71/24/513a99e58bb6b8465bae4d2d5e9dba8f0bef8179e3051ac414
  Created wheel for funcy: filename=funcy-1.14-py2.py3-none-any.whl size=32042 sha256=f7c6c5f1a69a605a0296bd3f29083bf29e84ee5096ac4bcbe6fe2b5e78f4fb84
  Stored in directory: /root/.cache/pip/wheels/20/5a/d8/1d875df03deae6f178dfdf70238cca33f948ef8a6f5209f2eb
Successfully built pyLDAvis funcy


In [ ]:
import pyLDAvis
import pyLDAvis.gensim
import warnings
import pickle

In [ ]:
def explore_topic(topic_number, topn=25):
    """
    accept a user-supplied topic number and
    print out a formatted list of the top terms
    """
        
    print('{:20} {}'.format(u'term', u'frequency') + u'\n')

    for term, frequency in lda_model.show_topic(topic_number, topn=25):
        print(u'{:20} {:.3f}'.format(term, round(frequency, 3)))

In [ ]:
lda_model.show_topic(12, topn=4)

[('complete', 0.14297357),
 ('available', 0.12505423),
 ('british', 0.120589375),
 ('clean', 0.044251725)]

In [ ]:
lda_model.show_topic(3)

[('happy', 0.1705421),
 ('dead', 0.105783686),
 ('ok', 0.09425617),
 ('incredible', 0.08605453),
 ('recent', 0.044075318),
 ('natural', 0.035464678),
 ('moral', 0.026114538),
 ('innocent', 0.022102661),
 ('much', 0.018696427),
 ('i', 0.016980525)]

In [ ]:
explore_topic(topic_number=2)

In [ ]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=corpus, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

In [ ]:
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)

In [ ]:
pyLDAvis.display(LDAvis_prepared)